<a id='pred_top'>

# Predict auction price

Try several models and improve predicition accuracy

## Model fitting

- Linear fits  
  1. [Simple linear fit](#pred_model_1)  
     No cross validation. Observations with missing values are dropped.
  2. [Dependent values scaled](#pred_model_2)  
     Dependent value here is _prices_.
  3. [Partial data](#pred_model_3)  
     Only young cars
- Multiple linear regression models  
  1. [MLR fit without imputation](#pred_model_4)  
  2. [With imputation](#pred_model_5)  
  3. [Include categorical features](#pred_model_6)  
  4. [Lasso regularization](#pred_model_7)  
  5. [include engineered features](#pred_model_8)

## Results

- [Model performance](#pred_accuracies)
- [Save best model](#pred_save_model) **TODO**  
  This is not implemented yet. Some preprocessing functions are not handled well with `pickle`.
- [Predictions](#pred_predict)
     
  

In [1]:
import sys
import os
import re
import json

In [2]:
with open('../assets/drz-settings-current.json', 'r') as fid:
    cfg = json.load(fid)
print(cfg['AUCTION'])

if cfg['AUCTION']['kind'] == 'opbod':
    raise NotImplementedError
    
OPBOD = cfg['AUCTION']['kind'] == 'opbod'
AUCTION_ID = cfg['AUCTION']['id']
DATA_DIR = cfg['FILE_LOCATION']['data_dir']
RESULTS_DIR = cfg['FILE_LOCATION']['report_dir']
VERBOSE = int(cfg['GENERAL']['verbose'])
SAVE_METHOD = cfg['GENERAL']['save_method']


{'kind': 'opbod', 'id': '2025-0601', 'date': '20250607'}


NotImplementedError: 

In [ ]:
if SAVE_METHOD == 'skip_when_exist':
    do_save = lambda fn: not(os.path.isfile(fn))
elif SAVE_METHOD == 'always_overwrite':
    do_save = lambda _: True
elif SAVE_METHOD == 'skip_save':
    do_save = lambda _: False
else:
    raise NotImplementedError(f'SAVE_METHOD: {SAVE_METHOD} not implemented')

In [ ]:
SAVE_METHOD

In [ ]:
TAG_SINGLE = "nbconvert_instruction:remove_single_output"


In [ ]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import seaborn as sns

In [ ]:
pd.__version__, np.__version__


In [ ]:
# set figure defaults (needs to be in cell seperate from import sns)
plt.style.use([
    'default',
    f"{cfg['FILE_LOCATION']['app_dir']}/assets/movshon.mplstyle",
    f"{cfg['FILE_LOCATION']['app_dir']}/assets/context-notebook.mplstyle"
])

# Load data

In [ ]:
fn = f'{DATA_DIR}/cars-for-ml.pkl'
print(fn)
df = pd.read_pickle(fn)
print(df.shape)

# time deltas
sel = (df.dtypes == 'timedelta64[ns]') | (df.columns == 'age_at_import')
df.loc[:, sel] = df.loc[:, sel].applymap(lambda x: x.days).astype('Float64')
# nullable boolean
sel = df.dtypes == 'boolean'
df.loc[:,sel] = df.loc[:,sel].astype('O').fillna(np.nan)
# int to float
df.price = df.price.astype('Float64')
# categories
cat_columns = ['brand', 'model', 'fuel', 'body_type','color', 'energy_label', 'fourwd', 'automatic_gearbox', 'under_survey']
# numerical
num_columns = list(np.setdiff1d(df.columns, cat_columns + ['price']))
df.loc[:, num_columns] = df.loc[:, num_columns].astype('Float64')

# Factorized categorical values
fld = 'energy_label'
# replace empty with NaN creates factor '-1'
v, idx = pd.factorize(df[fld].replace({'': np.nan}), sort=True)
# convert '-1' back to NaN
v = v.astype(float)
v[v==-1] = np.nan
# Store in dataframe
new_col = 'converted_' + fld
df[new_col] = v
# update list
num_columns += [new_col]
cat_columns.remove(fld)
print('\nCategorical field [{}] is converted to sequential numbers with: '.format(fld), end='\n\t')
print(*['{} <'.format(c) for c in idx], end='\n\n')

# convert boolean to string
for fld in ['fourwd', 'automatic_gearbox', 'under_survey']:
    if fld not in df.columns:
        print(f'!{fld} not in data!. Skip for now')
        continue
    new_col = fld
    # # update list
    # cat_columns += [new_col]
    # cat_columns.remove(fld)
    replace_dict = {
        '': '', 
        True: 'y', 
        False: 'n'
    }
    df[new_col] = df[fld].astype('O').replace(replace_dict)
    print('\nBoolean field [{}] is converted to numbers according to: '.format(fld), end='\n')
    print(*['\t"{}" -> {} ({})\n'.format(k,v, type(v)) for k,v in replace_dict.items()], end='\n\n')

# convert integer to float and replace -1
for fld in ['number_of_cylinders', 'number_of_doors', 'number_of_gears', 'number_of_seats']:
    if fld not in df.columns:
        print(f'!{fld} not in data!. Skip for now')
        continue
    new_col = fld
    replace_dict = {
        -1: np.nan, 
    }
    df[new_col] = df[fld].replace(replace_dict).astype(float)

# convert empty string to NaN
for fld in ['brand', 'model', 'fuel', 'body_type', 'color', 'fourwd']:
    if fld not in df.columns:
        print(f'!{fld} not in data!. Skip for now')
        continue
    new_col = fld
    replace_dict = {
        '': np.nan, 
    }
    df[new_col] = df[fld].replace(replace_dict)

# translate Dutch to English
fld = 'color'
new_col = fld
# # update list
# cat_columns += [new_col]
# cat_columns.remove(fld)
replace_dict = {
    '': 'missing', 
    'BLAUW': 'Blue',
    'ROOD': 'Red',
    'GROEN': 'Green',
    'GRIJS': 'Gray',
    'WIT': 'White',
    'ZWART': 'Black',
    'BEIGE': 'Beige',
    'BRUIN': 'Brown',
    'ROSE': 'Pink',
    'GEEL': 'Yellow',
    'CREME': 'Creme',
    'ORANJE': 'Orange',
    'PAARS': 'Purple'
}
df[new_col] = df[fld].replace(replace_dict)
print('\nField [{}] is converted according to: '.format(fld), end='\n')
print(*['\t"{}" -> {} ({})\n'.format(k,v, type(v)) for k,v in replace_dict.items()], end='\n\n')

# reporting
try:
    print('Categorical:', len(cat_columns))
    [print('\t[{:2.0f}] {:s}'.format(i+1, c)) for i,c in enumerate(df[cat_columns].columns)]
    print('Numercial:', len(num_columns))
    [print('\t[{:2.0f}] {:s}'.format(i+1, c)) for i,c in enumerate(df[num_columns].columns)]
    print('Last lot in data set:\n\t{}'.format(df.index[-1]))
except:
    cat_columns = [c for c in cat_columns if c in df.columns]
    num_columns = [c for c in num_columns if c in df.columns]    
    print('! not all fields are in data !. Skip for now')

In [ ]:
# Store model results in dictonary: Instantiate empty dict
models = dict()

In [ ]:
def split_shelve_vars():
    import types
    to_shelve = {}
    not_to_shelve = {}

    # loop over global variables (within this function is ignored)
    for var,val in globals().items():
        
        # skip variables based on names
        if re.match('^_(\d+|(i+\d*))$', var) is not None:
            not_to_shelve[var] = '-n'
            continue
        if re.match('^_+$', var) is not None:
            not_to_shelve[var] = '---'
            continue
        if var in ('_dh', '_ih', '_oh'):
            not_to_shelve[var] = '-dio'
            continue
        if var in ('In', 'Out'):
            not_to_shelve[var] = '-io'
            continue
        if var in ('__doc__', '__loader__', '__name__', '__package__', '__session__', '__spec__'):
            not_to_shelve[var] = val
            continue
        
        # skip built-ins and modules
        if isinstance(globals()[var], (types.ModuleType, types.BuiltinFunctionType, types.FunctionType)):
            not_to_shelve[var] = type(globals()[var])
            continue
        else:
            pass
            #print(globals()[var].__class__, var)

        # store
        to_shelve[var] = val
        
    return not_to_shelve, to_shelve
    
drop,keep = split_shelve_vars()
list(keep.keys())

In [ ]:
keep.pop('get_ipython')
keep.pop('exit')
keep.pop('quit')
keep.pop('fid')

In [ ]:
import shelve
from inspect import getsource
import types


In [ ]:
with shelve.open('./predict-price.shelve', flag='n') as slf:
    for k in [
        'cfg',
        #'OPBOD',
        #'AUCTION_ID',
        #'DATA_DIR',
        'RESULTS_DIR',
        #'VERBOSE',
        #'SAVE_METHOD',
        #'TAG_SINGLE',
        #'fn',
        'df',
        'cat_columns',
        'num_columns',
        'models',
        'do_save']:
        print(k)
        if k in keep:
            v = keep[k]
        else:
            v = globals()[k]
            if isinstance(v, types.FunctionType):
                src = getsource(v).strip()
                cnt = len([k for k in slf.keys() if k.startswith('def')])
                k = f'def{cnt}:'
                v = src
            
        slf[k] = v


In [ ]:
for m in range(1,9):
    nb = f"./predict-price-model{m}.ipynb"
    display({'text/html':f'<HR><h3>Running {nb}</h3><hr>'}, raw=True)
    %run {nb}

In [ ]:
with shelve.open('./predict-price.shelve', flag='r') as slf:
    for k,v in slf.items():
        print(k)
        globals()[k] = v
        if re.match('def\d+:', k) is not None:
            print(v)
            exec(v) 

In [ ]:
# Best model
model_name = 'MLR added features'

# Display prediction errors

x_sample = df.dropna(subset=['price']).iloc[:,1:]
y_sample = df.dropna(subset=['price']).iloc[:,0]
# Add features
x_sample.loc[:,'usage_intensity'] = x_sample.odometer / x_sample.age
x_sample.loc[:,'classic'] = x_sample.age > 25*365
x_sample.loc[:,'classic'] = x_sample.loc[:,'classic'].astype('O').replace({True:'y', False:'n'})
#x_sample.loc[:,'youngtimer'] = (x_sample.age > 15*365) & (x_sample.age <= 25*365)
#x_sample.loc[:,'youngtimer'].replace({True:'y', False:'n'}, inplace=True)
x_sample[pd.isna(x_sample)] = np.nan
# predict again
y_sample_pred = models[model_name]['model'].predict(x_sample) 

x_sample['price'] = y_sample
x_sample['prediction_error'] = y_sample_pred - y_sample
x_sample['prediction_error_fraction'] = y_sample_pred/y_sample
x_sample['prediction_error_log'] = np.log10(x_sample.prediction_error_fraction)
x_sample['prediction_error_abslog'] = np.abs(np.log10(x_sample.prediction_error_fraction))
x_sample['prediction'] = y_sample_pred
x_sample['age_y'] = x_sample.age/365


# Note some are close to perfect, because they are in training set and are unique in brand etc
print(f'best predictons of [{model_name}] model')
display(x_sample.sort_values(by='prediction_error_abslog').head(16).T)
print('worst predictions')
display(x_sample.sort_values(by='prediction_error_abslog').tail(16).T)
print('largest underestimate')
display(x_sample.sort_values(by='prediction_error').head(16).T)
print('largest overestimate')
display(x_sample.sort_values(by='prediction_error').tail(16).T)
print('worst prediction recent auction')
is_last_auction = x_sample.index.str.startswith('-'.join(x_sample.index[-1].split('-')[:2]))
display(x_sample[is_last_auction].sort_values(by='prediction_error_abslog').tail(8).T)

plt.figure(figsize=[8,8])
plt.plot(x_sample.age_y, x_sample.prediction_error_log, color='k', marker='s', markeredgecolor = (0, 0, 0, 0), markerfacecolor = (0, 0, 0, 1), linestyle='None', ms=1)
plt.axhline(0, lw=2, linestyle='--', color ='k')
plt.xlabel('age [years]')
plt.ylabel('prediction error [log of fraction]\n(positive: prediction overestimates)')
plt.show()

## Model accuracies

In [ ]:
# plot R^2

# counter for x-offset
c=0

# figure
fig = plt.figure(figsize=[4,2])
ax = fig.gca()
xs = ys = fs = np.empty(0)

# loop over all models
for name,res in models.items():

    c+=1 # x-offset

    if name == 'linear regression no cv':
        # No cv, so only one value. Make it a list of one for type consistency
        k = 'R^2'
        rsq = [res[k]]
    
    else: 
        k = 'cv R^2'
        rsq = res[k]
        
    if 'n betas effective' in res:
        ndf = res['n betas effective']
    elif 'betas' in res:
        ndf = len(res['betas'])
    elif 'n effective features' in res:
        ndf = res['n effective features']
        
    # add r-squares and offset to vectors
    ys = np.concatenate([ys, rsq])
    xs = np.concatenate([xs, np.ones_like(rsq) * c])
    fs = np.concatenate([fs, [ndf]])

# actual plotting
sns.swarmplot(x=xs, y=ys, ax=ax, hue=None)
ax.bar(range(0,len(models)), [res['R^2'] for res in models.values()], width=0.8, fc='none')
for x,ndf in enumerate(fs):
    if ndf is None:
        continue
    if x == 0:
        s = f'd.f.: {ndf:.0f}'
    else:
        s = f'{ndf:.0f}'
    ax.text(x, 1, s, ha='center')
# prettify
ax.set_xticks(range(0,len(models)))
ax.set_xticklabels(labels=list(models.keys()), rotation=45, va='top', ha='right', style='italic')
ax.set_ylim(bottom=0, top=+1)
ax.set_title('Model performance\n', style='italic')
ax.set_ylabel('Coefficient of determination\n($R^2$)', style='italic')
ax.xaxis.set_tick_params(which='minor', bottom=False)

# save
file_name = f"{RESULTS_DIR}/model-performance.png"
if True | do_save(file_name): # always save
    print(file_name)
    with plt.style.context(f"{cfg['FILE_LOCATION']['app_dir']}/assets/context-paper.mplstyle"):
        plt.savefig(file_name, bbox_inches='tight', transparent=False)
else:
    plt.show()
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

In [ ]:
# plot data

# loop over all models
for model_name in models.keys():
    print(model_name)
    res = models[model_name]
    features = num_columns.copy()
    
    # model specific adjustments
    if (model_name == 'linear regression log price') \
    or (model_name == 'linear regression log price young'):
        yX = df.loc[:,['price', 'age']].dropna()
        X = yX.iloc[:,1]
        y = yX.iloc[:,0]
        X[pd.isna(X)] = np.nan
        # log price is used
        y = np.log10(y)
        # unit
        unit = '(log[EUR])'
    elif (model_name == 'MLR reduced observations') \
    or (model_name == 'MLR impute median'):
        yX = df.dropna(subset=['price'] + features).loc[:,['price'] + features]
        X = yX.iloc[:,1:]
        y = np.log10(yX.iloc[:,0])
        X[pd.isna(X)] = np.nan
        unit = '(log[EUR])'
    elif (model_name == 'MLR with categorical') \
    or (model_name == 'MLR Lasso'):
        yX = df.dropna(subset=['price']).copy()
        X = yX.iloc[:,1:]
        y = yX.iloc[:,0]
        X[pd.isna(X)] = np.nan
        unit = '(EUR)'
    elif (model_name == 'MLR added features'):
        yX = df.dropna(subset=['price']).copy()
        X = yX.iloc[:,1:]
        y = yX.iloc[:,0]
        X[pd.isna(X)] = np.nan
        unit = '(EUR)'
        X.loc[:,'usage_intensity'] = X.odometer / X.age
        X.loc[:,'classic'] = X.age > 25*365
        X.loc[:,'classic'] = X.loc[:,'classic'].astype('O').replace({True:'y', False:'n'})
        X[pd.isna(X)] = np.nan
    elif (model_name == 'Decision Tree Regression'):
        yX = df.dropna(subset=['price']).copy()
        X = yX.iloc[:,1:]
        y = yX.iloc[:,0]
        X[pd.isna(X)] = np.nan
        unit = '(EUR)'
        X.loc[:,'usage_intensity'] = X.odometer / X.age
        X.loc[:,'classic'] = X.age > 25*365
        X.loc[:,'classic'] = X.loc[:,'classic'].astype('O').replace({True:+1, False:-1, np.nan:0}, inplace=True)
        for col in ['fourwd', 'under_survey', 'automatic_gearbox']:
            X.loc[:, col].replace({'n':0, 'y':1}, inplace=True)
            
    else:
        # all original data
        yX = df.loc[:,['price', 'age']].dropna()
        X = yX.iloc[:,1]
        y = yX.iloc[:,0]
        X[pd.isna(X)] = np.nan
        unit = '(EUR)'
    
    if X.ndim != 1:
        n_feat = X.shape[1]
    else:
        n_feat = 1
        
    if not model_name in ('MLR with categorical', 'MLR Lasso', 'MLR added features', 'Decision Tree Regression'):
        # needed for .predict
        X = np.array(X).reshape(-1,n_feat)
        y = np.array(y).reshape(-1,1)
    
    # predict all data
    y_pred = res['model'].predict(X)
    if max(y) < 10:
        rmse = np.sqrt(np.mean(((10**y)-(10**y_pred))**2))
    else:
        rmse = np.sqrt(np.mean((y-y_pred)**2))
    print(rmse)

    # actual plotting
    fig,axs = plt.subplots(nrows=2, ncols=1, figsize=[8,8])
    
    # data
    axs[0].plot(y, y_pred, marker=',', linestyle='None')
    # error
    axs[1].plot(y, y_pred-y, marker=',', linestyle='None')
    
    # axis equal for top
    if (model_name == 'MLR with categorical') or (model_name == 'MLR Lasso') or (model_name == 'MLR added features') or (model_name == 'Decision Tree Regression'):
        axs[0].set_xscale('log')
        axs[0].set_yscale('log')
        axs[1].set_xscale('log')
    axs[0].set_aspect(1)
    # store limits
    yl = axs[0].get_ylim()
    xl_top = axs[0].get_xlim()
    xl_bot = axs[1].get_xlim()
    xl = [np.max([xl_top[0], xl_bot[0]]), np.min([xl_top[1], xl_bot[1]])]
    # plot unity line and 0 error
    unity_line = [np.max([xl[0], yl[0]]), np.min([xl[1], yl[1]])]
    axs[0].plot(unity_line, unity_line, '-k', linewidth=2)
    axs[1].plot(xl, [0, 0], '-k', linewidth=2)
    # reset limits
    axs[0].set_xlim(xl)
    axs[1].set_xlim(xl)

    # make equal size panels
    # Note: sharex did not work
    bb=axs[0].get_position(False)
    rect_top = bb.bounds
    bb=axs[1].get_position(False)
    rect_bot = bb.bounds
    rect = list(rect_bot)
    rect[0] = rect_top[0]
    rect[2] = rect_top[2]
    axs[1].set_position(rect)
    
    # labeling
    fig.suptitle('{}\nrmse: EUR {:.0f}'.format(model_name,rmse), style='italic')
    axs[1].set_xlabel('Real price ' + unit, style='italic')
    axs[0].set_ylabel('Predicted price\n' + unit, style='italic')
    axs[1].set_ylabel('Prediction error\n' + unit, style='italic')
    
    # save
    file_name = f"{RESULTS_DIR}/{model_name.replace(' ','_')}-accuracy.png"
    if True | do_save(file_name): # always save
        print(file_name)
        with plt.style.context(f"{cfg['FILE_LOCATION']['app_dir']}/assets/context-paper.mplstyle"):
            plt.savefig(file_name, bbox_inches='tight', transparent=False)
    else:
        plt.show()
        print(f'Skip. {file_name} exists or saving is disabled in settings.')